In [1]:
import pandas as pd
pd.set_option('display.max_rows', 5)
import tkinter as tk
from tkinter import filedialog

# 處理天線數據

In [2]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [3]:
rf = pd.read_excel(file_path, sheet_name='RF')

In [4]:
rf.columns

Index(['營運中心', '狀態', '狀態2', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '交期變更',
       '客戶名稱', '業務代號', '負責業務', '客戶訂單', '客戶訂單項次', '業務部門', '銷售類', '交貨方式', '料號',
       '產品分類', '品名', '銷售單位', '幣別', '數量', '已出數量', '未出數', '單價', '金額', '訂單匯率',
       '金額*訂單匯率', '客戶料號', '其它分群一碼(接單業務)', '帳款客戶編號', '業務名稱', '出通單號', '代理商編號',
       '發票號碼', 'INVOICE號碼', '客戶希交日', 'TERM', '備註', 'C.C', '客戶專案名稱', '原始希交日',
       'HSF編號(報價單)', 'HSF編號(最新)', '產品類別', '出通單日', '出貨單號', '舊系統單號', '車用',
       '用戶端車型', '天線擺放位置', 'REBATE', '集團匯率', '集團匯率*金額', '資料所有者', '資料修改者',
       '出貨日期', '送貨客戶編號', '送貨客戶名稱', '數量(舊系統)', '來源單據', 'BU', '年份', '月', '組別2',
       'RF/元件', '客戶名稱品名', '產業別對照表201109'],
      dtype='object')

In [5]:
keep_columns = ['狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '交期變更', '客戶名稱', '負責業務', '交貨方式', '產品分類', '品名', '幣別', '數量', '已出數量', '未出數', '銷售單位',
                    '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日', 'TERM', '出通單號', '客戶訂單', '客戶訂單項次']

In [6]:
rf = rf[keep_columns]

In [7]:
origin_data_num = rf.shape[0]

In [8]:
rf = rf.drop_duplicates(['銷售單號', '銷售項次', '客戶訂單', '客戶訂單項次'], keep='last')

In [9]:
droped_data_num = rf.shape[0]

In [10]:
print("總共有", origin_data_num - droped_data_num, "筆重複數據")

總共有 0 筆重複數據


In [11]:
rf_unit_change = rf[rf['銷售單位'] == 'KPCS']
rf_keep_unit = rf[~(rf['銷售單位'] == 'KPCS')]
rf_unit_change['數量'] = rf_unit_change['數量'].map(lambda x: x * 1000)
rf_unit_change['已出數量'] = rf_unit_change['已出數量'].map(lambda x: x * 1000)
rf_unit_change['未出數'] = rf_unit_change['未出數'].map(lambda x: x * 1000)
rf_unit_change['單價'] = rf_unit_change['單價'].map(lambda x: x / 1000)

rf = pd.concat([rf_unit_change, rf_keep_unit], axis=0)

append_dict = {0: 'Category', 1: 'BG', 2: 'Subcategory', 3: 'Group', 10: '預交年份', 11: '預交月份', }
for k, v in append_dict.items():
    rf.insert(k, v, value=None)

rf = rf.drop(['銷售單位', '客戶訂單', '客戶訂單項次'], axis=1)

<ipython-input-11-b29bd10e5fc7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_unit_change['數量'] = rf_unit_change['數量'].map(lambda x: x * 1000)
<ipython-input-11-b29bd10e5fc7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_unit_change['已出數量'] = rf_unit_change['已出數量'].map(lambda x: x * 1000)
<ipython-input-11-b29bd10e5fc7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [12]:
columns = ['Category', 'BG', 'Subcategory', 'Group', '狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '預交年份', '預交月份', '交期變更',
               '客戶名稱', '負責業務', '交貨方式', '產品分類', '品名', '幣別', '數量', '已出數量', '未出數', '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日',
               'TERM', '出通單號']

In [13]:
result = rf.reindex(columns=columns)

In [14]:
result['負責業務'] = result['負責業務'].map(lambda x: '許凱智' if x == '楊婉芬' or x == '周彥宏' else x)
result = result[result['負責業務'].isin(['鄭里緗', '墨漢雷迪', '許凱智'])]

In [15]:
result['數量'] = result['數量'].astype(int)
result['已出數量'] = result['已出數量'].astype(int)
result['未出數'] = result['未出數'].astype(int)
result['幣別'] = result['幣別'].map(lambda x: 'NTD' if x == 'TWD' else x)


In [16]:
result.to_excel(r'C:\Users\kaihsu\Desktop\業績總表\2021_rf_clean.xlsx', index=False)

# 2021元件

In [18]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [19]:
zhunan = pd.read_excel(file_path, header=3)

In [20]:
zhunan = zhunan[:-2]

In [21]:
zhunan.columns

Index(['營運中心', '狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '交期變更', '客戶名稱',
       '業務代號', '負責業務', '客戶訂單', '客戶訂單項次', '業務部門', '銷售類', '交貨方式', '料號', '產品分類',
       '品名', '銷售單位', '幣別', '數量', '已出數量', '未出數', '單價', '金額', '訂單匯率', '金額*訂單匯率',
       '客戶料號', '其它分群一碼(接單業務)', '帳款客戶編號', '業務名稱', '出通單號', '代理商編號', '發票號碼',
       'INVOICE號碼', '客戶希交日', 'TERM', '備註', 'C.C', '客戶專案名稱', '原始希交日',
       'HSF編號(報價單)', 'HSF編號(最新)', '產品類別', '出通單日', '出貨單號', '舊系統單號', '車用',
       '用戶端車型', '天線擺放位置', 'REBATE', '集團匯率', '集團匯率*金額', '資料所有者', '資料修改者',
       '出貨日期', '送貨客戶編號', '送貨客戶名稱', '數量(舊系統)', '來源單據', 'BU'],
      dtype='object')

In [22]:
keep_columns = ['狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '交期變更', '客戶名稱', '負責業務', '交貨方式',
                '產品分類', '品名', '幣別', '數量', '銷售單位', '已出數量', '未出數', 
                '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日', 'TERM', '出通單號', '客戶訂單', '客戶訂單項次']

In [23]:
zhunan = zhunan[keep_columns]

In [24]:
origin_data = zhunan.shape[0]

In [25]:
zhunan = zhunan[zhunan['客戶名稱'] != 'INPAQ']

In [26]:
zhunan = zhunan.drop_duplicates(subset=['銷售單號', '銷售項次', '客戶訂單', '客戶訂單項次'], keep='last')

In [27]:
dropped_data = zhunan.shape[0]
print("總共有", origin_data - dropped_data, "筆重複數據。")

總共有 297 筆重複數據。


In [28]:
zhunan_unit_change = zhunan[zhunan['銷售單位'] == 'KPCS']
zhunan_keep_unit = zhunan[~(zhunan['銷售單位'] == 'KPCS')]
zhunan_unit_change['數量'] = zhunan_unit_change['數量'].map(lambda x: x * 1000)
zhunan_unit_change['已出數量'] = zhunan_unit_change['已出數量'].map(lambda x: x * 1000)
zhunan_unit_change['未出數'] = zhunan_unit_change['未出數'].map(lambda x: x * 1000)
zhunan_unit_change['單價'] = zhunan_unit_change['單價'].map(lambda x: x / 1000)

zhunan = pd.concat([zhunan_unit_change, zhunan_keep_unit], axis=0)

append_dict = {0: 'Category', 1: 'BG', 2: 'Subcategory', 3: 'Group', 10: '預交年份', 11: '預交月份', }
for k, v in append_dict.items():
    zhunan.insert(k, v, value=None)

zhunan = zhunan.drop(['銷售單位', '客戶訂單', '客戶訂單項次'], axis=1)

<ipython-input-28-f08a456f96c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zhunan_unit_change['數量'] = zhunan_unit_change['數量'].map(lambda x: x * 1000)
<ipython-input-28-f08a456f96c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zhunan_unit_change['已出數量'] = zhunan_unit_change['已出數量'].map(lambda x: x * 1000)
<ipython-input-28-f08a456f96c7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [29]:
columns = ['Category', 'BG', 'Subcategory', 'Group', '狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '預交年份', '預交月份', '交期變更',
               '客戶名稱', '負責業務', '交貨方式', '產品分類', '品名', '幣別', '數量', '已出數量', '未出數', '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日',
               'TERM', '出通單號']
result = zhunan.reindex(columns=columns)

In [30]:
result = result[result["負責業務"].isin(['鄭里緗', '許凱智', '墨漢雷迪'])]

In [31]:
result['數量'] = result['數量'].astype(int)
result['已出數量'] = result['已出數量'].astype(int)
result['未出數'] = result['未出數'].astype(int)
result['幣別'] = result['幣別'].map(lambda x: 'NTD' if x == 'TWD' else x)
# result['集團匯率*金額'] = result['集團匯率*金額'].astype(int)
result.to_excel(r'C:\Users\kaihsu\Desktop\業績總表\2021_component_clean.xlsx', index=False)

# 2021小顧無錫廠

In [32]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(file_path)

In [ ]:
df.columns

In [ ]:
keep_columns = ['状态', '销售单号', '销售项次', '销售月份', '开单日期', '预交日期', '交期变更', '送货客户名称', '负责业务', '交货方式', '产品分类', '品名', '币别', '数量', '已出数量', '未出数', '销售单位',
                    '单价', '集团汇率', '集团汇率*金额', '客户料号', '客户希交日', 'TERM', '出通单号', '客户订单', '客户订单项次']
df = df[keep_columns]

In [ ]:
df.columns = ['狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '交期變更', '客戶名稱', '負責業務', '交貨方式', '產品分類', '品名', '幣別', '數量', '已出數量', '未出數', '銷售單位',
                    '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日', 'TERM', '出通單號', '客戶訂單', '客戶訂單項次']

In [ ]:
origin_data = df.shape[0]

In [ ]:
df = df.drop_duplicates(['銷售單號', '銷售項次', '客戶訂單', '客戶訂單項次'], keep='last')

In [ ]:
dropped_data = df.shape[0]
print("總共有", origin_data - dropped_data, "筆重複數據")

In [ ]:
df_unit_change = df[df['銷售單位'] == 'KPCS']
df_keep_unit = df[~(df['銷售單位'] == 'KPCS')]
df_unit_change['數量'] = df_unit_change['數量'].map(lambda x: x * 1000)
df_unit_change['已出數量'] = df_unit_change['已出數量'].map(lambda x: x * 1000)
df_unit_change['未出數'] = df_unit_change['未出數'].map(lambda x: x * 1000)
df_unit_change['單價'] = df_unit_change['單價'].map(lambda x: x / 1000)

df = pd.concat([df_unit_change, df_keep_unit], axis=0)

append_dict = {0: 'Category', 1: 'BG', 2: 'Subcategory', 3: 'Group', 10: '預交年份', 11: '預交月份', }
for k, v in append_dict.items():
    df.insert(k, v, value=None)

df = df.drop(['銷售單位', '客戶訂單', '客戶訂單項次'], axis=1)

columns = ['Category', 'BG', 'Subcategory', 'Group', '狀態', '銷售單號', '銷售項次', '銷售月份', '開單日期', '預交日期', '預交年份', '預交月份', '交期變更',
               '客戶名稱', '負責業務', '交貨方式', '產品分類', '品名', '幣別', '數量', '已出數量', '未出數', '單價', '集團匯率', '集團匯率*金額', '客戶料號', '客戶希交日',
               'TERM', '出通單號']
result = df.reindex(columns=columns)

df['數量'] = df['數量'].astype(int)
df['已出數量'] = df['已出數量'].astype(int)
df['未出數'] = df['未出數'].astype(int)
# df['集團匯率*金額'] = df['集團匯率*金額'].astype(int)


In [ ]:
df['負責業務'] = df['負責業務'].map(lambda x: '鄭里緗' if x == '沈思明' or x == '鄭裡緗' else x)

In [ ]:
df["負責業務"].unique()

In [ ]:
df = df[df["客戶名稱"] == "LF.瑞侃"]

In [ ]:
df.to_excel(r'C:\Users\kaihsu\Desktop\業績總表\2021_小顧_clean.xlsx', index=False)

### 將所有的合併為一份

In [33]:
root = tk.Tk()
root.withdraw()
file_paths = filedialog.askopenfilenames()

In [34]:
files_list = []
for file in file_paths:
    files_list.append(pd.read_excel(file))
result = pd.concat(files_list, axis=0)
result.to_excel(r'C:\Users\kaihsu\Desktop\業績總表\匯總數據_final.xlsx', index=False)

## 將Weekly轉出csv匯入github給dashboard使用

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [ ]:
root.withdraw()
export_path = filedialog.askdirectory()

In [ ]:
destination = export_path + "/weekly_report.csv"

In [ ]:
df.to_csv(destination, index=False)